# Import Libraries

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

In [3]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
from huggingface_hub import login
login(token=user_secrets.get_secret("HF_READ"))
login(token=user_secrets.get_secret("HF_WRITE"))

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful
The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [4]:
from transformers import AutoModelForCausalLM, BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(load_in_4bit=True,
                                        bnb_4bit_compute_dtype=torch.bfloat16,
                                        bnb_4bit_quant_type="nf4",
                                        bnb_4bit_use_double_quant=True)

In [5]:
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=quantization_config, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_id)

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [6]:
memory_footprint =model.get_memory_footprint()/1024**3
print(f"memory_footprint:{memory_footprint} GB")

memory_footprint:5.2075347900390625 GB


In [7]:
model.config.quantization_config.to_dict()

{'quant_method': <QuantizationMethod.BITS_AND_BYTES: 'bitsandbytes'>,
 '_load_in_8bit': False,
 '_load_in_4bit': True,
 'llm_int8_threshold': 6.0,
 'llm_int8_skip_modules': None,
 'llm_int8_enable_fp32_cpu_offload': False,
 'llm_int8_has_fp16_weight': False,
 'bnb_4bit_quant_type': 'nf4',
 'bnb_4bit_use_double_quant': True,
 'bnb_4bit_compute_dtype': 'bfloat16',
 'bnb_4bit_quant_storage': 'uint8',
 'load_in_4bit': True,
 'load_in_8bit': False}

In [8]:
model.push_to_hub("SwastikM/Meta-Llama-3-8B-Instruct_bitsandbytes_4bit")

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/SwastikM/Meta-Llama-3-8B-Instruct_bitsandbytes_4bit/commit/96af67edf79112fde82e66389fad8125fb9f4c3d', commit_message='Upload LlamaForCausalLM', commit_description='', oid='96af67edf79112fde82e66389fad8125fb9f4c3d', pr_url=None, pr_revision=None, pr_num=None)

In [9]:
messages = [
    {"role": "system", "content": "You are a Coder."},
    {"role": "user", "content": "How to ctrate a list in Python?"}
]


In [10]:
input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

In [11]:
input_ids

tensor([[128000, 128006,   9125, 128007,    271,   2675,    527,    264,    356,
           4414,     13, 128009, 128006,    882, 128007,    271,   4438,    311,
          51738,    349,    264,   1160,    304,  13325,     30, 128009, 128006,
          78191, 128007,    271]], device='cuda:0')

In [12]:
outputs = model.generate(
    input_ids,
    max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=False,
    temperature=0.0
)

/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:540: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:545: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token.As a consequence, you may observe unexpec

In [13]:
response = outputs[0][input_ids.shape[-1]:]
print(tokenizer.decode(response, skip_special_tokens=True))

In Python, you can create a list in several ways:

1. Using the `list()` function:
```
my_list = list()
```
This creates an empty list.

2. Using square brackets `[]`:
```
my_list = []
```
This also creates an empty list.

3. Using the `list()` function with an iterable (such as a string or a tuple):
```
my_list = list("hello")
print(my_list)  # Output: ['h', 'e', 'l', 'l', 'o']
```
4. Using the `list()` function with a range of numbers:
```
my_list = list(range(1, 6))
print(my_list)  # Output: [1, 2, 3, 4, 5]
```
5. Using the `list()` function with a dictionary:
```
my_dict = {"a": 1, "b": 2, "c": 3}
my_list = list(my_dict.keys())
print(my_list)  # Output: ['a', 'b', 'c']
```
Note that in Python, lists are mutable, meaning you can add, remove, or modify elements after creating the list.

For example:
``


# Prepare Dataset

In [14]:
from datasets import load_dataset
datasets = load_dataset("Salesforce/xlam-function-calling-60k")

Generating train split:   0%|          | 0/60000 [00:00<?, ? examples/s]

In [15]:
datasets['train'][0]

{'query': 'Where can I find live giveaways for beta access and games?',
 'id': 0,
 'answers': '[{"name": "live_giveaways_by_type", "arguments": {"type": "beta"}}, {"name": "live_giveaways_by_type", "arguments": {"type": "game"}}]',
 'tools': '[{"name": "live_giveaways_by_type", "description": "Retrieve live giveaways from the GamerPower API based on the specified type.", "parameters": {"type": {"description": "The type of giveaways to retrieve (e.g., game, loot, beta).", "type": "str", "default": "game"}}}]'}

In [16]:
raw_dataset = datasets['train'].train_test_split(test_size=0.1,seed=1)

In [17]:
raw_dataset['train'] = raw_dataset['train'].select([*range(2000)])
raw_dataset['test'] = raw_dataset['train'].select([*range(100)])

In [18]:
raw_dataset

DatasetDict({
    train: Dataset({
        features: ['query', 'id', 'answers', 'tools'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['query', 'id', 'answers', 'tools'],
        num_rows: 100
    })
})

In [19]:
print("query : ",raw_dataset['train'][0]['query']+"\n")
print("tools : ",raw_dataset['train'][0]['tools']+"\n")
print("answers : ",raw_dataset['train'][0]['answers']+"\n")

query :  Get a random Counter-Strike skin from the 'Sniper' class and search for E-Sports players, teams, and tournaments related to 'Counter-Strike: Global Offensive'.

tools :  [{"name": "random_skin_from_class", "description": "Fetches information about a random Counter-Strike skin from a specified class using the CS Skin API.", "parameters": {"class_name": {"description": "The class of Counter-Strike skin to retrieve, e.g., \"Rifle\".", "type": "str", "default": "Knife"}}}, {"name": "search", "description": "Search for E-Sports players, teams, and tournaments by providing a search term.", "parameters": {"term": {"description": "The search term used to find E-Sports entities.", "type": "str", "default": "ATK"}}}]

answers :  [{"name": "random_skin_from_class", "arguments": {"class_name": "Sniper"}}, {"name": "search", "arguments": {"term": "Counter-Strike: Global Offensive"}}]



In [20]:
x1 = {"role": "system", "content": """You are a APIGen Function Calling Tool. You will br provided with a user query and associated tools for answering the query.
    query (string): The query or problem statement.
    tools (array): An array of available tools that can be used to solve the query.
    Each tool is represented as an object with the following properties:
        name (string): The name of the tool.
        description (string): A brief description of what the tool does.
        parameters (object): An object representing the parameters required by the tool.
            Each parameter is represented as a key-value pair, where the key is the parameter name and the value is an object with the following properties:
                type (string): The data type of the parameter (e.g., "int", "float", "list").
                description (string): A brief description of the parameter.
                required (boolean): Indicates whether the parameter is required or optional.
    You will provide the Answer array.
        Answers array provides the specific tool and arguments used to generate each answer."""}
x2 = {"role": "user", "content": None}
x3 = {"role": "assistant", "content": None}
prompts = [x1,x2,x3]
prompts

[{'role': 'system',
  'content': 'You are a APIGen Function Calling Tool. You will br provided with a user query and associated tools for answering the query.\n    query (string): The query or problem statement.\n    tools (array): An array of available tools that can be used to solve the query.\n    Each tool is represented as an object with the following properties:\n        name (string): The name of the tool.\n        description (string): A brief description of what the tool does.\n        parameters (object): An object representing the parameters required by the tool.\n            Each parameter is represented as a key-value pair, where the key is the parameter name and the value is an object with the following properties:\n                type (string): The data type of the parameter (e.g., "int", "float", "list").\n                description (string): A brief description of the parameter.\n                required (boolean): Indicates whether the parameter is required or optio

In [21]:
user_template = 'Query: {Q} Tools: {T}'
response_template = '{A}'

In [22]:
def preprocess_function(examples):
    query = examples['query']
    tools = examples['tools']
    answers = examples['answers']
    model_inputs_list=[]
    model_inputs = {}
    for Q,T,A  in zip(query,tools,answers):
        x2['content'] = f'{user_template.format(Q=Q,T=T)}'
        x3['content'] = f'{response_template.format(A=A)}'
        prompt = [x1,x2,x3]
        input_ids = tokenizer.apply_chat_template(
            prompt,
            add_generation_prompt=False,
            return_tensors="pt",
            max_length=1024,
            truncation=True
            )
        model_inputs_list.append(input_ids[0])
    model_inputs["input_ids"] = model_inputs_list
    return model_inputs

In [23]:
tokenized_datasets = raw_dataset.map(preprocess_function, batched=True,remove_columns=datasets["train"].column_names)

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

# Data Collator

In [24]:
from transformers import DataCollatorForLanguageModeling

tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

# Optimizer

In [25]:
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=2e-5)

In [26]:
model.config

LlamaConfig {
  "_name_or_path": "meta-llama/Meta-Llama-3-8B-Instruct",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 128000,
  "eos_token_id": 128009,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 14336,
  "max_position_embeddings": 8192,
  "mlp_bias": false,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 8,
  "pretraining_tp": 1,
  "quantization_config": {
    "_load_in_4bit": true,
    "_load_in_8bit": false,
    "bnb_4bit_compute_dtype": "bfloat16",
    "bnb_4bit_quant_storage": "uint8",
    "bnb_4bit_quant_type": "nf4",
    "bnb_4bit_use_double_quant": true,
    "llm_int8_enable_fp32_cpu_offload": false,
    "llm_int8_has_fp16_weight": false,
    "llm_int8_skip_modules": null,
    "llm_int8_threshold": 6.0,
    "load_in_4bit": true,
    "load_in_8bit": false,
    "quant_method": "bitsandbytes"
  

# Preapre Model for PEFT

In [27]:
from peft import LoraConfig, TaskType

In [28]:
peft_config = LoraConfig(inference_mode=False,
                         r=8,
                         lora_alpha=32,
                         lora_dropout=0.1,task_type=TaskType.CAUSAL_LM)

In [29]:
from peft import get_peft_model,prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)
lora_model = get_peft_model(model, peft_config)
lora_model.print_trainable_parameters()

trainable params: 3,407,872 || all params: 8,033,669,120 || trainable%: 0.0424


# Train Model

In [30]:
from transformers import Trainer, TrainingArguments

args = TrainingArguments(
    output_dir="finetuned_model",
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    eval_strategy="epoch",
    logging_steps=10,
    gradient_accumulation_steps=2,
    num_train_epochs=1,
    warmup_steps=8,
    lr_scheduler_type="linear",
    learning_rate=2e-4,
    save_strategy='epoch',
    fp16=True,
    report_to='none'

)

trainer = Trainer(
    model=lora_model,
    tokenizer=tokenizer,
    args=args,
    data_collator=data_collator,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
)

lora_model.config.use_cache = False

In [31]:
trainer.train()

/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.461000,0.376943


TrainOutput(global_step=1000, training_loss=0.43582794761657717, metrics={'train_runtime': 10627.6168, 'train_samples_per_second': 0.188, 'train_steps_per_second': 0.094, 'total_flos': 5.368369170635981e+16, 'train_loss': 0.43582794761657717, 'epoch': 1.0})

In [32]:
lora_model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 4096)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaSdpaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=Fal

In [33]:
lora_model.save_pretrained("output_dir")

In [34]:
lora_model.push_to_hub("SwastikM/Meta-Llama3-8B-Chat-Adapter")

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/13.6M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/SwastikM/Meta-Llama3-8B-Chat-Adapter/commit/b8a5ce9dffd5ce5cbd1a1f79a0dcb59a9e0d43f3', commit_message='Upload model', commit_description='', oid='b8a5ce9dffd5ce5cbd1a1f79a0dcb59a9e0d43f3', pr_url=None, pr_revision=None, pr_num=None)

# Inference

In [35]:
x1 = {"role": "system", "content": """You are a APIGen Function Calling Tool. You will br provided with a user query and associated tools for answering the query.
    query (string): The query or problem statement.
    tools (array): An array of available tools that can be used to solve the query.
    Each tool is represented as an object with the following properties:
        name (string): The name of the tool.
        description (string): A brief description of what the tool does.
        parameters (object): An object representing the parameters required by the tool.
            Each parameter is represented as a key-value pair, where the key is the parameter name and the value is an object with the following properties:
                type (string): The data type of the parameter (e.g., "int", "float", "list").
                description (string): A brief description of the parameter.
                required (boolean): Indicates whether the parameter is required or optional.
    You will provide the Answer array.
        Answers array provides the specific tool and arguments used to generate each answer."""}
x2 = {"role": "user", "content": None}
x3 = {"role": "assistant", "content": None}
user_template = 'Query: {Q} Tools: {T}'
response_template = '{A}'
Q = "Where can I find live giveaways for beta access and games?"
T = """[{"name": "live_giveaways_by_type", "description": "Retrieve live giveaways from the GamerPower API based on the specified type.", "parameters": {"type": {"description": "The type of giveaways to retrieve (e.g., game, loot, beta).", "type": "str", "default": "game"}}}]"""

In [36]:

x2['content'] = f'{user_template.format(Q=Q,T=T)}'
prompts = [x1,x2]
input_ids = tokenizer.apply_chat_template(
    prompts,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]


In [37]:
outputs = lora_model.generate(
    input_ids,
    max_new_tokens=256,
    eos_token_id=terminators
)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


In [38]:
response = outputs[0][input_ids.shape[-1]:]
print(tokenizer.decode(response, skip_special_tokens=True))

[{"name": "live_giveaways_by_type", "arguments": {"type": "beta"}}, {"name": "live_giveaways_by_type", "arguments": {"type": "game"}}]
